In [11]:
import pandas as pd
import datetime
import re

In [37]:
df = pd.read_excel("./data/france_dataset.xlsx")
cols = df.columns

In [38]:
df['date_hospital_admi'] = pd.to_datetime(df['date_hospital_admi'])
df['date_ICU_admi'] = pd.to_datetime(df['date_ICU_admi'])
df['date_hospital_end'] = pd.to_datetime(df['date_hospital_end'], errors='coerce')
df['date_death'] = pd.to_datetime(df['date_death'], errors='coerce')

In [24]:
df_base = df[['id', 'date_hospital_admi', 'date_hospital_end', 'date_death', 'ICU_DC']]
df_base['end-admi'] = (df_base['date_hospital_end'] - df_base['date_hospital_admi']).dt.days

<ipython-input-24-4c95223f4acf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base['end-admi'] = (df_base['date_hospital_end'] - df_base['date_hospital_admi']).dt.days


In [26]:
# Admission data: HR_min, HR_max, SBP_min, SBP_max, DBP_min, DBP_max, SAPSII, RR, GCS
df_admi = df.filter([ 'HR_min', 'HR_max', 'SBP_min', 'SBP_max', 'DBP_min', 'DBP_max', 'SAPSII', 'RR', 'GCS'])

In [46]:
all_cols = df.columns.to_list()
for e in ['HR_min', 'HR_max', 'SBP_min', 'SBP_max', 'DBP_min', 'DBP_max', 'SAPSII', 'RR', 'GCS']:
    all_cols.remove(e)
for e in ['id', 'date_hospital_admi', 'date_hospital_end', 'date_death', 'ICU_DC']:
    all_cols.remove(e)
det_cols = all_cols

In [29]:
df_dete = df.iloc[:,23:]

In [39]:
df.dropna(subset=['ICU_DC'])

,id,HR_min,HR_max,SBP_min,SBP_max,DBP_min,DBP_max,SAPSII,RR,GCS,...,ldh_j1,ldh_j2,ldh_j3,ldh_j4,ldh_j5,L_pourc_J1,L_pourc_J2,L_pourc_J3,L_pourc_J4,L_pourc_J5
0,49,61,94,93,190,66,132,27,28,15,...,.,465.00,457.00,412.00,362.0,25.0000,.,.,.,.
1,52,85,105,90,107,56,73,31,28,15,...,652.0,900.00,.,.,.,.,13.3333,.,.,.
2,55,81,170,101,154,48,66,54,20,15,...,498.0,505.00,.,.,596.0,.,.,.,.,.
3,56,79,124,105,132,40,65,27,.,15,...,568.0,.,.,.,.,4.1667,.,.,.,.
4,58,77,90,110,166,54,80,47,.,15,...,455.0,450.00,535.00,.,497.0,10.0000,.,.,4.1176,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,415,87,104,106,137,67,85,33,31,15,...,503.0,.,.,.,.,4.7619,.,6.1224,.,5.2632
174,420,64,89,109,176,69,97,37,32,13,...,466.0,552.00,599.00,.,.,1.9417,.,.,6.1856,6.2500
175,421,52,77,92,159,51,87,48,25,15,...,550.0,.,.,.,.,15.7895,13.2075,.,.,.
176,424,84,97,129,152,77,85,33,26,15,...,579.0,.,.,.,.,6.5041,.,8.6667,.,2.9630


In [44]:
df_dete

0
id  dete_num          
0   Death_D          0
    Death_D          0
    ddimere_J   1300.0
    ddimere_J   1400.0
    ddimere_J   1200.0
...                ...
177 L_pourc_J   6.7164
    L_pourc_J  18.1818
    L_pourc_J  34.4262
    L_pourc_J  26.0870
    L_pourc_J  30.4762

[3916 rows x 1 columns]

In [43]:
df_dete = pd.DataFrame(df_dete.rename(columns=lambda x: re.sub('\d', '', x)).stack(level=0))
df_dete = df_dete.rename(columns={'crp_J': 'CRP', 'ldh_j': 'LDH', 'L_pourc_J': 'L%'})
df_dete.index.names = ['id', 'dete_num']

df_ = df_dete.join(df_base, on='id').drop(columns=['id'])
    
# Select the data and rename it
df_ = df_.dropna(subset=['ICU_DC'])
df_ = df_.groupby(level=0).last()
df_ = df_.reset_index(drop=True)

df_ = df_.rename(columns={'ICU_DC': 'Survival/death'})
df_ = df_[features_en+['Survival/death']]
df_['Survival/death'] = df_['Survival/death'].astype(int)

TypeError: expected string or bytes-like object

In [15]:
features_en = ['LDH', 'CRP', 'L%']
# Detection data
df_dete = df.iloc[:,21:] #df.iloc[:, 19:]
##df_dete = df_dete.rename(columns=lambda x: int(re.sub('Day ', '', x)), level=0)
#df_dete = df_dete.rename(columns=lambda x: int(re.sub('Death_D', '', x)), level=0)
df_dete = pd.DataFrame(df_dete.rename(columns=lambda x: re.sub('\d', '', x)).stack(level=0))
df_dete = df_dete.rename(columns={'crp_J': 'CRP', 'ldh_j': 'LDH', 'L_pourc_J': 'L%'})
df_dete.index.names = ['id', 'dete_num']

df = df_dete.join(df_base, on='id').drop(columns=['id'])
    
    # Select the data and rename it
df = df.dropna(subset=['ICU_DC'])
df = df.groupby(level=0).last()
df = df.reset_index(drop=True)
df = df.rename(columns={'ICU_DC': 'Survival/death'})

df = df[features_en+['Survival/death']]
df['Class'] = 'FO (last)'
df['Survival/death'] = df['Survival/death'].astype(int)

IndexError: list index out of range

In [12]:
data

,id,HR_min,HR_max,SBP_min,SBP_max,DBP_min,DBP_max,SAPSII,RR,GCS,...,ldh_j1,ldh_j2,ldh_j3,ldh_j4,ldh_j5,L_pourc_J1,L_pourc_J2,L_pourc_J3,L_pourc_J4,L_pourc_J5
0,49,61,94,93,190,66,132,27,28,15,...,.,465.00,457.00,412.00,362.0,25.0000,.,.,.,.
1,52,85,105,90,107,56,73,31,28,15,...,652.0,900.00,.,.,.,.,13.3333,.,.,.
2,55,81,170,101,154,48,66,54,20,15,...,498.0,505.00,.,.,596.0,.,.,.,.,.
3,56,79,124,105,132,40,65,27,.,15,...,568.0,.,.,.,.,4.1667,.,.,.,.
4,58,77,90,110,166,54,80,47,.,15,...,455.0,450.00,535.00,.,497.0,10.0000,.,.,4.1176,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,415,87,104,106,137,67,85,33,31,15,...,503.0,.,.,.,.,4.7619,.,6.1224,.,5.2632
174,420,64,89,109,176,69,97,37,32,13,...,466.0,552.00,599.00,.,.,1.9417,.,.,6.1856,6.2500
175,421,52,77,92,159,51,87,48,25,15,...,550.0,.,.,.,.,15.7895,13.2075,.,.,.
176,424,84,97,129,152,77,85,33,26,15,...,579.0,.,.,.,.,6.5041,.,8.6667,.,2.9630


In [11]:
def load_fr_data():
    """
    Load France data
    Because there is no detection time, so take the last detection result
    """
    df = pd.read_excel('./data/france_dataset.xlsx')
    #idx = df[('Unnamed: 0_level_0', 'id')]
    #idx.name = 'id'
    #df = df.set_index(idx)
    
    # Base data
    df_base = df.iloc[:, :10]
    df_base = df_base.droplevel(level=0, axis=1)
    df_base = df_base[['id', 'date_hospital_admi', 'date_hospital_end', 'date_death', 'ICU_DC']]
    df_base['date_hospital_admi'] = pd.to_datetime(df_base['date_hospital_admi'], format="%d/%m/%Y")
    df_base['date_hospital_end']  = pd.to_datetime(df_base['date_hospital_end'], format="%d/%m/%Y")
    df_base['date_death']         = pd.to_datetime(df_base['date_death'], format="%d/%m/%Y")
    df_base['end-admi'] = (df_base['date_hospital_end'] - df_base['date_hospital_admi']).dt.days
    
    # Admission data: HR_min, HR_max, SBP_min, SBP_max, DBP_min, DBP_max, SAPSII, RR, GCS
    df_admi = df.iloc[:, 10:19]
    
    # Detection data
    df_dete = df.iloc[:, 19:]
    df_dete = df_dete.rename(columns=lambda x: int(re.sub('Day ', '', x)), level=0)
    df_dete = df_dete.rename(columns=lambda x: re.sub('\d', '', x), level=1).stack(level=0)
    df_dete = df_dete.rename(columns={'CRPt': 'CRP', 'LDHt': 'LDH', 'L_perct': 'L%'})
    df_dete.index.names = ['id', 'dete_num']

    df = df_dete.join(df_base, on='id').drop(columns=['id'])
    
    # Select the data and rename it
    df = df.dropna(subset=['ICU_DC'])
    df = df.groupby(level=0).last()
    df = df.reset_index(drop=True)
    df = df.rename(columns={'ICU_DC': 'Survival/death'})
    df = df[features_en+['Survival/death']]
    df['Class'] = 'FO (last)'
    df['Survival/death'] = df['Survival/death'].astype(int)

    return df

load_fr_data()

ValueError: Cannot remove 1 levels from an index with 1 levels: at least one level must be left.

# kolumny czasowo-śmierciowe

In [3]:
df =data[['date_hospital_admi','date_ICU_admi','date_death', 'Death_D7', 'Death_D28']][data['date_death'] != "."]
df

,date_hospital_admi,date_ICU_admi,date_death,Death_D7,Death_D28
1,2020-03-25,2020-03-25,2020-04-01 00:00:00,1,1
14,2020-03-10,2020-03-10,2020-03-12 00:00:00,0,1
15,2020-03-12,2020-03-13,2020-03-17 00:00:00,0,1
22,2020-03-13,2020-03-15,2020-03-24 00:00:00,0,0
23,2020-03-11,2020-03-15,2020-03-25 00:00:00,0,1
...,...,...,...,...,...
157,2020-03-26,2020-03-26,2020-04-12 00:00:00,0,0
159,2020-03-28,2020-03-28,2020-04-22 00:00:00,0,0
169,2020-03-26,2020-03-26,2020-04-14 00:00:00,0,0
170,2020-03-30,2020-04-01,2020-04-07 00:00:00,0,1


In [4]:
df['date_hospital_admi'] = pd.to_datetime(df['date_hospital_admi'])
df['date_ICU_admi'] = pd.to_datetime(df['date_ICU_admi'])
df['date_death'] = pd.to_datetime(df['date_death'])

In [5]:
df["from_door_to_death"] = df['date_death'] - df['date_hospital_admi']

In [6]:
df

,date_hospital_admi,date_ICU_admi,date_death,Death_D7,Death_D28,from_door_to_death
1,2020-03-25,2020-03-25,2020-04-01,1,1,7 days
14,2020-03-10,2020-03-10,2020-03-12,0,1,2 days
15,2020-03-12,2020-03-13,2020-03-17,0,1,5 days
22,2020-03-13,2020-03-15,2020-03-24,0,0,11 days
23,2020-03-11,2020-03-15,2020-03-25,0,1,14 days
...,...,...,...,...,...,...
157,2020-03-26,2020-03-26,2020-04-12,0,0,17 days
159,2020-03-28,2020-03-28,2020-04-22,0,0,25 days
169,2020-03-26,2020-03-26,2020-04-14,0,0,19 days
170,2020-03-30,2020-04-01,2020-04-07,0,1,8 days


# Martwi bez daty śmierci - jeszcze ok

In [46]:
dead = (data["Death_D7"] != 0) | (data["Death_D28"] != 0)
incorrect = data[(data["date_death"] == ".") & dead].filter(["date_death","Death_D7", "Death_D28" ])
incorrect

,date_death,Death_D7,Death_D28
2,.,0,1
3,.,0,1
4,.,0,1
7,.,1,1
8,.,0,1
12,.,0,1
20,.,0,1
21,.,1,1
24,.,0,1
27,.,1,1


In [47]:
incorrect.count()

date_death    42
Death_D7      42
Death_D28     42
dtype: int64

# Martwi z datą śmierci, ale bez flagi śmieci

In [64]:
not_dead = (df["Death_D7"] == 0) & (df["Death_D28"] == 0)
incorrect2 = df[(df["date_death"] != ".") & not_dead].filter(["date_death","Death_D7", "Death_D28", "from_door_to_death" ])
incorrect2

,date_death,Death_D7,Death_D28,from_door_to_death
22,2020-03-24,0,0,11 days
30,2020-04-07,0,0,7 days
32,2020-04-05,0,0,11 days
35,2020-04-22,0,0,16 days
42,2020-02-14,0,0,16 days
44,2020-03-24,0,0,23 days
46,2020-03-19,0,0,12 days
48,2020-03-29,0,0,24 days
49,2020-04-01,0,0,26 days
56,2020-03-30,0,0,23 days


In [65]:
incorrect2.count()

date_death            43
Death_D7              43
Death_D28             43
from_door_to_death    43
dtype: int64

# dodajmy ramy czasowe 

In [69]:
inc2_range = incorrect2[incorrect2["from_door_to_death"] < datetime.timedelta(days=28)]
inc2_range

,date_death,Death_D7,Death_D28,from_door_to_death
22,2020-03-24,0,0,11 days
30,2020-04-07,0,0,7 days
32,2020-04-05,0,0,11 days
35,2020-04-22,0,0,16 days
42,2020-02-14,0,0,16 days
44,2020-03-24,0,0,23 days
46,2020-03-19,0,0,12 days
48,2020-03-29,0,0,24 days
49,2020-04-01,0,0,26 days
56,2020-03-30,0,0,23 days


In [71]:
inc2_range.count()

date_death            38
Death_D7              38
Death_D28             38
from_door_to_death    38
dtype: int64